# 1.Tổng quan People and Parcel Share a Ride
- N người, đánh số i từ [1,N]. Đón người i ở điểm i, trả ở điểm i+N+M. Nếu đón người i phải có đường đi trực tiếp từ i->i+N+M
- M hàng, đánh số j từ [1,M]. Nhận hàng j ở điểm j+N, trả ở điểm j+2N+M. Nếu nhận hàng j thì trong đường đi về sau phải đi qua j+2N+M.
- K xe, đánh số k từ [0,K-1]. Khởi hành ở 0, kết thúc sẽ ở 0. 
- parcel_quantities: danh sách số lượng hàng mỗi loại.
- taxi_capacities: danh sách trọng tải hàng mỗi xe.
- Yêu cầu tối ưu: min(max(route length of each taxi))

# 2. Thiết lập nhập dữ liệu N,M,K,PQ,TC

In [55]:
from ortools.linear_solver import pywraplp

def read_input():
    #
    N, M, K = map(int, input().split())
    
    # nhập vào danh sách số lượng hàng mỗi loại
    parcel_quantities = list(map(int, input().split()))
    
    # Read taxi capacities  
    taxi_capacities = list(map(int, input().split()))
    
    # Read distance matrix
    distance_matrix = []
    for _ in range(2 * N + 2 * M + 1):
        row = list(map(int, input().split()))
        distance_matrix.append(row)
        
    return N, M, K, parcel_quantities, taxi_capacities, distance_matrix



# 3. Thiết lập biến quyết định
- X[i,j,k]: i thuộc [0,2N+2M],: xe k có đi từ i đến j. Với 0 là điểm depot
- Y[i,k]: i thuộc [0,N-1]: xe k có chở người i, Đón khách i ở i+1, trả khách i ở i+N+1+M
- Z[i,k]: i thuộc [0,M-1]: Xe k có chở hàng loại i, Đón hàng i ở i+1+N, trả hàng ở i + 1 + N + N + M

In [56]:
def create_variables(solver, N, M, K):
   
    # X[i,j,k] = 1 if taxi k travels from i to j
    X = {}
    for i in range(0, 2 * N + 2 * M + 1):
        for j in range(0, 2 * N + 2 * M + 1):
            for k in range(K):
                X[i, j, k] = solver.BoolVar(f'X[{i},{j},{k}]')
    
    # Y[i,k] = 1 if passenger i is served by taxi k
    Y = {}
    for i in range(N):
        for k in range(K):
            Y[i, k] = solver.BoolVar(f'Y[{i},{k}]')
    
    # Z[i,k] = 1 if parcel i is carried by taxi k
    Z = {}
    for i in range(M):
        for k in range(K):
            Z[i, k] = solver.BoolVar(f'Z[{i},{k}]')
            
    # T[k] = total distance traveled by taxi k
    T = [solver.NumVar(0, solver.infinity(), f'T[{k}]') for k in range(K)]
    
    # D = maximum distance among all taxis
    D = solver.NumVar(0, solver.infinity(), 'D')
    
    return X, Y, Z, T, D

# Thiết lập ràng buộc
- mỗi hành khách phải được phục vụ bởi chỉ 1 taxi: tổng của Y[i,k] cho k chạy = 1
- nếu Y[i,k]==1 thì X[i+1, i + 1 + N + M, k] == Y[i, k]


In [78]:
def add_constraints(solver, N, M, K, X, Y, Z, T, D, parcel_quantities, taxi_capacities, distance_matrix):
    # Maximum distance constraint
    for k in range(K):
        solver.Add(T[k] <= D)

    # Each passenger must be served by exactly one taxi
    for i in range(N):
        solver.Add(sum(Y[i, k] for k in range(K)) == 1)
        # Link passenger pickup and dropoff
        for k in range(K):
            solver.Add(X[i+1, i+1 + N + M, k] == Y[i, k])

    # Taxi capacity constraints for parcels
    for k in range(K):
        solver.Add(sum(parcel_quantities[i] * Z[i, k] for i in range(M)) <= taxi_capacities[k])

    # Each parcel must be carried by exactly one taxi
    for i in range(M):
        solver.Add(sum(Z[i, k] for k in range(K)) == 1)
        # Link parcel pickup and delivery
        for k in range(K):
            solver.Add(sum(X[j, N+i+1, k] for j in range(2 * N + 2 * M + 1)) == Z[i, k])
            solver.Add(sum(X[N + i + 1, j, k] for j in range(2 * N + 2 * M + 1)) == Z[i, k])
            solver.Add(sum(X[j, 2*N + M + i + 1, k] for j in range(2 * N + 2 * M + 1)) == Z[i, k])
            solver.Add(sum(X[2*N + M + i + 1,j, k] for j in range(2 * N + 2 * M + 1)) == Z[i, k])

    # All taxis must start and end at depot
    for k in range(K):
        solver.Add(sum(X[0, j, k] for j in range(1, 2 * N + 2 * M + 1)) == 1)
        solver.Add(sum(X[i, 0, k] for i in range(1, 2 * N + 2 * M + 1)) == 1)

    # Flow conservation constraints
    for k in range(K):
        for j in range(2 * N + 2 * M + 1):
            solver.Add(sum(X[i, j, k] for i in range(2 * N + 2 * M + 1) if i != j) ==
                      sum(X[j, i, k] for i in range(2 * N + 2 * M + 1) if i != j))

    # Calculate total distance for each taxi
    for k in range(K):
        solver.Add(T[k] == sum(distance_matrix[i][j] * X[i, j, k]
                              for i in range(2 * N + 2 * M + 1)
                              for j in range(2 * N + 2 * M + 1)))

   # Ensure pickup before delivery for parcels (corrected logic)
   

    # Subtour elimination constraints using MTZ formulation
    # Biến U[i] đại diện thứ tự thăm của node i (1 <= i <= 2 * N + 2 * M)
    U = [solver.NumVar(0, 2 * N + 2 * M, f'U[{i}]') for i in range(1, 2 * N + 2 * M + 1)]

    for k in range(K):
        for i in range(1, 2 * N + 2 * M + 1):
            for j in range(1, 2 * N + 2 * M + 1):
                if i != j:
                    solver.Add(U[i - 1] - U[j - 1] + (2 * N + 2 * M) * X[i, j, k] <= (2 * N + 2 * M - 1))


    return solver


In [79]:
def extract_routes(X, K, N, M):
    routes = []
    for k in range(K):
        route = []
        current_point = 0  # Start at the depot
        visited = set([0])  # Mark the depot as visited

        while True:
            found_next = False
            for j in range(2 * N + 2 * M + 1):  # Check all nodes
                if X[current_point, j, k].solution_value() > 0.5 and j not in visited:
                    route.append(j)  # Add the next node to the route
                    visited.add(j)  # Mark the node as visited
                    current_point = j  # Move to the next node
                    found_next = True
                    break
            if not found_next:
                break  # No more nodes to visit, end the route
        routes.append(route)
    return routes


In [ ]:
def print_solution_matrix(X, K, N, M):
    for k in range(K):
        print(f"Taxi {k}:")
        matrix = []
        for i in range(2 * N + 2 * M + 1):
            row = []
            for j in range(2 * N + 2 * M + 1):
                row.append(int(X[i, j, k].solution_value()))  # Convert solution value to int for clarity
            matrix.append(row)
        for row in matrix:
            print(row)
        print("\n")  # Separate matrices for each taxi


In [ ]:
def main():
    # Create solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    N, M, K = map(int, input().split())
    parcel_quantities = list(map(int, input().split()))
    taxi_capacities = list(map(int, input().split()))
    distance_matrix = []
    for _ in range(2 * N + 2 * M + 1):
        distance_matrix.append(list(map(int, input().split())))
        
        
    for i in range(2*N+2*M+1):
        distance_matrix[i][i] = 100
    X, Y, Z, T, D = create_variables(solver, N, M, K)
    
    # Set objective: minimize maximum distance
    solver.Minimize(D)
    
    # Add constraints
    add_constraints(solver, N, M, K, X, Y, Z, T, D, parcel_quantities, taxi_capacities, distance_matrix)
    
    # Solve the problem
    status = solver.Solve()
    
    # Output results
    if status == pywraplp.Solver.OPTIMAL:
        print(K)
        routes = extract_routes(X, K, N, M)
        for route in routes:
            print(len(route) + 2)  # +2 for start and end depot
            print(f"0 {' '.join(map(str, route))} 0")
        print_solution_matrix(X, K, N, M)     
    else:
        print('No optimal solution found.')

main()